In [10]:
from bs4 import BeautifulSoup
import re 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy
from selenium.webdriver.common.proxy import ProxyType
import time
import sys
import pandas as pd
import requests
import threading
import concurrent.futures
from queue import Queue 


In [3]:
#把https://www.free-proxy-list.net/中的proxy list抓下來 並且做request測試能否上網（以抓取ppt頁面為測試行為）

#只抓頁面上欄位https為yes的proxy 
def collectProxy():
    proxy_url = 'https://www.free-proxy-list.net/'
    resp = requests.get(proxy_url)
    resp.encodingn = 'utf-8'
    text = resp.text
    table = pd.read_html(text)[0]
    table_https = table.loc[table['Https'] == "yes"] 
    
    #用ppt beauty版做抓取測試
    text_url = "https://www.ptt.cc/bbs/Beauty/index.html"
    test_proxy_lists = []
    good_proxy_q = Queue(maxsize=0) #放最終好的proxy ip 

    for n in range (0,len(table_https)):
        ip = table_https["IP Address"][table_https.index[n]]
        port = str(table_https["Port"][table_https.index[n]]).replace(".0","")
        full_ip = ip+":"+ port
        test_proxy_lists.append(full_ip) #把proxy ip data處理好放list裡面 
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:  #開10的thread來跑測試 
        future_to_storename = {executor.submit(testProxy, test_proxy): test_proxy for test_proxy in test_proxy_lists}

def testProxy(test_proxy):  
    try:
        proxies = {
            'https': "http://%s"%test_proxy,
        }

        resp = requests.get(text_url,proxies=proxies,timeout=10)
        if "看板 Beauty 文章列表 - 批踢踢實業坊" in resp.text:
            print("good proxy" + proxy)
            good_proxy_q.put(proxy)
        else:
            print("fail proxy:"+ proxy)

    except Exception as e:
        print("fail proxy:"+ proxy + "錯誤訊息：" + str(e)) 

        
if __name__ == '__main__':
    collectProxy()
    

In [3]:
table_https

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
2,171.99.204.151,8080.0,TH,Thailand,anonymous,no,yes,25 seconds ago
5,159.203.181.50,3128.0,US,United States,anonymous,no,yes,25 seconds ago
6,163.172.27.213,3128.0,GB,United Kingdom,elite proxy,no,yes,25 seconds ago
9,89.236.17.108,3128.0,SE,Sweden,elite proxy,no,yes,25 seconds ago
11,188.93.110.26,8080.0,RU,Russian Federation,anonymous,no,yes,25 seconds ago
13,185.93.3.123,8080.0,ES,Spain,anonymous,no,yes,25 seconds ago
16,151.80.140.233,54566.0,FR,France,anonymous,no,yes,25 seconds ago
18,187.35.137.97,3128.0,BR,Brazil,anonymous,no,yes,25 seconds ago
20,133.130.103.208,8080.0,JP,Japan,elite proxy,no,yes,25 seconds ago
21,200.95.174.19,3128.0,BR,Brazil,elite proxy,no,yes,25 seconds ago


In [12]:
good_proxy_lists

['89.236.17.108:3128',
 '188.93.110.26:8080',
 '185.93.3.123:8080',
 '151.80.140.233:54566',
 '133.130.103.208:8080',
 '35.196.26.166:3128',
 '192.116.142.153:8080',
 '188.254.78.108:8080',
 '89.22.175.42:8080',
 '203.129.126.226:8080',
 '154.119.50.246:53281',
 '194.182.64.67:3128',
 '216.177.233.181:8080',
 '61.220.26.97:80',
 '47.206.51.67:8080',
 '190.60.4.198:3128',
 '5.197.231.174:8080']

In [14]:
len(good_proxy_lists)

17